# Cuaderno 6: Planificación de la producción (Lot Sizing)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\ZZ}{\mathbb{Z}_+}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$

En los problemas de planificación de la producción (*lot sizing*) se busca determinar los niveles óptimos de producción de uno o varios bienes dentro de un ciclo productivo, considerando restricciones como la demanda, la capacidad, los costos de producción, los costos de almacenamiento, entre otras.

Consideraremos, a manera de ejemplo, la planificación de los niveles óptimos de producción de una empresa dedicada a la producción de alimento balanceado, sujeta a las siguientes condiciones:

* El plan de producción se organiza sobre un horizonte de tiempo de un año, dividido en un conjunto $T= \tabulatedset{1, \ldots, 12}$ de períodos mensuales. 
* Si en un período $t \in T$ se toma la decisión de producir, se incurre en un
  costo fijo igual a $c_t$ (en USD). 
* Para cada período $t \in T$, se conocen además la demanda estimada de balanceado en el mercado $d_t$ (en toneladas) y el costo unitario de su producción $p_t$ (en USD / tonelada). 
* En cada período se debe satisfacer toda la demanda estimada. Si en un período el nivel de producción supera a la demanda, es posible almacenar el producto excedente a un costo unitario igual a $h_t$ (USD/tonelada). 
* La producción se realiza por lotes de una tonelada de alimento balanceado. No es posible producir fracciones de un lote.
  
El problema consiste en determinar los niveles de producción mensuales que permitan satisfacer toda la demanda al menor costo posible. Para formular este problema como un programa lineal entero, definimos las siguientes variables de decisión:

* variables enteras $x_t, \, t \in T,$ que indiquen la cantidad de lotes a producir en el período $t$;
* variables binarias $y_t, \, t \in T,$ tales que $y_t = 1$ si y solamente si se toma la decisión producir en el período $t$;
* variables enteras $z_t, \, t \in T,$ que indiquen la cantidad de lotes de producto a almacenar del período $t$ al período $t+1$.

Con estas definiciones, el problema puede ser formulado como el siguiente programa lineal entero:

\begin{align*}
\min &\sum_{t \in T} (p_t x_t + c_t y_t + h_t z_t)\\ 
& \mbox{s.r.}\\
& x_1 = d_1 + z_1,\\
& x_t + z_{t-1} = d_t + z_t, \quad \forall t \in T \setminus \{1\}, \\
& x_t \leq M y_t, \quad \forall t \in T, \\
& x_t, z_t \in \ZZ, y_t \in \{0, 1\}, \quad \forall t \in T.
\end{align*}


Donde $M$ es una constante suficientemente grande como para que $x_t \leq M$ se cumpla en cualquier solución factible. Podemos fijar, por ejemplo, $M:= \sum_{t \in T} d_t$.

La función objetivo tiene tres componentes que miden, para cada período, los costos variables de producción, los costos fijos de producción y los costos de almacenamiento.

Las dos primeras familias de restricciones establecen el balance entre la cantidad producida, la demanda y la cantidad almacenada: en cada período $t \in T \setminus\{1\}$, la suma entre la cantidad producida en $t$ más la cantidad almacenada desde el período $t-1$ debe ser igual a la suma de la demanda más la cantidad que se almacena para el período $t+1$. Para el primer período, no existe cantidad almacenada previamente. Observar que la no negatividad de las variables $z_t$ implica que el nivel de producción en cada período debe ser suficientemente alto como para cubrir la demanda.

La tercera familia de restricciones contiene restricciones de enforzamiento que indican que $x_t$ puede tomar valores mayores a cero, solamente cuando $y_t=1$, es decir, cuando se toma la decisión de producir en el período $t$.

Vamos a implementar este programa utilizando la interfaz Python de Gurobi.

Definimos primero el conjunto $T$ y los parámetros $p$, $c$, $h$, $d$ y $M$:

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Conjuntos y parámetros del modelo
# El diccionario con los datos tiene el formato {t : (c_t, p_t, h_t, d_t)}
# Costos fijos, costos de producción, costos de almacenamiento y demandas:
T, c, p, h, d = gp.multidict({
    1 : (400, 40, 10, 20),   2 : (450, 47, 12, 20),  3 : (400, 38, 10, 25),
    4 : (470, 45,  9, 25),   5 : (450, 47, 12, 25),  6 : (400, 42, 10, 20),
    7 : (350, 33, 10, 20),   8 : (400, 39, 12, 18),  9 : (450, 45, 10, 18),
   10 : (450, 47, 12, 20),  11 : (500, 50, 15, 25), 12 : (500, 55, 15, 27)})

    
# Constante igual a la suma de todas las demandas
M = gp.quicksum(d)

# Conjunto T \ {1}
T2 = [t for t in T if not t==1]

print("T= {}".format(T))
print("T2= {}".format(T2))
print("c= {}".format(c))
print("p= {}".format(p))
print("h= {}".format(h))
print("d= {}".format(d))
print("M= {}".format(M))

Definimos ahora el objeto modelo y las variables del modelo:

In [ ]:
m = gp.Model('lot-sizing')

# número de lotes a producir
x = m.addVars(T, vtype = GRB.INTEGER, name="x")

# se produce / no se produce en cada periodo
y = m.addVars(T, vtype = GRB.BINARY, name="y")

# número de lotes a almacenar
z = m.addVars(T, vtype = GRB.INTEGER, name="z")

Construimos la función objetivo a partir de sus tres términos:

In [ ]:
# costos variables de producción
c_produccion = x.prod(p, '*')

# costos fijos de producción
c_fijo = y.prod(c, '*')

# costos de almacenamiento
almacenamiento = z.prod(h, '*')

m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

Finalmente, implementamos las restricciones del modelo:
1. Balance entre produccion, almacenamiento y demanda.

In [ ]:
# Balance primer periodo
m.addConstr(x[1] - z[1] == d[1], "balance[1]")

# Balance periodos siguientes
m.addConstrs((x[t] + z[t-1] - z[t] == d[t] for t in T2), "balance")

2. Solamente puede producirse cuando $y_t =1$.

In [ ]:
# y[t]=0 => x[t]=0
m.addConstrs((x[t] <= M*y[t] for t in T), "enforz_xy")

Resolvemos el modelo:

In [ ]:
m.optimize()

Mostramos la solución: producción, almacenamiento y demanda en cada período. Notar que puede utilizarse el método `getAtrr` de la clase modelo para consultar los diferentes atributos de toda una familia de variables del modelo (almacenadas en un `tupledict`). En particular, el atributo `x` retorna el valor de las variables en la solución óptima.

In [ ]:
# Extraer valores de las soluciones
vx = m.getAttr('x', x)
vy = m.getAttr('x', y)
vz = m.getAttr('x', z)

# Mostrar tabla con los valores
print('Plan óptimo de producción:')
print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
for t in T:
    print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[t], vy[t], vz[t], d[t]))
    
# Otro método para lo mismo:
print('Plan óptimo de producción:')
print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
for t in T:
    print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, x[t].x, y[t].x, z[t].x, d[t]))

## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Curso de implementación de programas lineales enteros
# Ejemplo: Modelo de planificación de la producción (lot sizing)
# EPN (2022)

import gurobipy as gp
from gurobipy import GRB

try:
    # Conjuntos y parámetros del modelo
    # Costos fijos, costos de producción, costos de almacenamiento y demandas:
    T, c, p, h, d = gp.multidict({
        1 : (400, 40, 10, 20),   2 : (450, 47, 12, 20),  3 : (400, 38, 10, 25),
        4 : (470, 45,  9, 25),   5 : (450, 47, 12, 25),  6 : (400, 42, 10, 20),
        7 : (350, 33, 10, 20),   8 : (400, 39, 12, 18),  9 : (450, 45, 10, 18),
       10 : (450, 47, 12, 20),  11 : (500, 50, 15, 25), 12 : (500, 55, 15, 27)})

    
    # Constante igual a la suma de todas las demandas
    M = gp.quicksum(d)

    # Conjunto T \ {1}
    T2 = [t for t in T if not t==1]    
    
    # Crear el objeto modelo
    m = gp.Model('lot-sizing')

    # Variables de decisión
    # lotes a producir
    x = m.addVars(T, vtype = GRB.INTEGER, name="x")

    # se produce / no se produce en cada período
    y = m.addVars(T, vtype = GRB.BINARY, name="y")

    # lotes a almacenar
    z = m.addVars(T, vtype = GRB.INTEGER, name="z")    

    # Función objetivo
    # costos variables de producción
    c_produccion = x.prod(p, '*')

    # costos fijos de producción
    c_fijo = y.prod(c, '*')

    # costos de almacenamiento
    almacenamiento = z.prod(h, '*')

    m.setObjective(c_produccion + c_fijo + almacenamiento, GRB.MINIMIZE)

    # Restricciones
    # Balance primer período
    m.addConstr(x[1] - z[1] == d[1], "balance[1]")

    # Balance períodos siguientes
    m.addConstrs((x[t] + z[t-1] - z[t] == d[t] for t in T2), "balance")
    
    # y[t]=0 => x[t]=0
    m.addConstrs((x[t] <= M*y[t] for t in T), "sinc_xy")    
    
    # Resolver el modelo
    m.optimize()
    
    # Mostrar solucion
    # Extraer valores de las soluciones
    vx = m.getAttr('x', x)
    vy = m.getAttr('x', y)
    vz = m.getAttr('x', z)

    # Mostrar tabla con los valores
    print('Plan óptimo de producción:')
    print('t\t\tx_t\t\ty_t\t\tz_t\t\td_t')
    for t in T:
        print('{}\t\t{}\t\t{}\t\t{}\t\t{}'.format(t, vx[t], vy[t], vz[t], d[t]))        
except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')